# Qdrant Tools Registry:

This notebook demonstrates how to register and wrap the Qdrant search tool for agent use, by importing the implementation from the qdrant_tools folder.

In [12]:
# Ensure src/ is in sys.path so 'tools' can be imported
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(TOOLS_PATH)


if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src


In [13]:
"""
Qdrant Tools Registry: Central place to register and wrap Qdrant-related tools for agent use.
"""
from langchain_openai import OpenAIEmbeddings
from langchain.tools import Tool

# Import the QdrantSearchTool implementation from the actual server tool script
from tools.qdrant_tools.qdrant_server_tool import QdrantSearchTool


In [14]:
# Utility: get collection name from config/loader if needed
from utils.qdrant_client_loader import get_qdrant_collection_name
COLLECTION_NAME = get_qdrant_collection_name()


# Instantiate the tool
embedding_model = OpenAIEmbeddings()
qdrant_search_tool_instance = QdrantSearchTool(
    host="localhost",
    port=6333,
    collection_name=COLLECTION_NAME,
    embedding_model=embedding_model
)

def wrapped_qdrant_search(inputs: dict) -> list:
    import json
    if isinstance(inputs, str):
        try:
            inputs = json.loads(inputs)
        except Exception:
            raise ValueError("Tool input is not a valid JSON dictionary.")
    query = inputs.get("query", "")
    filters = inputs.get("filters", None)
    k = inputs.get("k", 5)
    print(f"\n[DEBUG] Query: {query}")
    print(f"[DEBUG] Filters: {filters}")
    print(f"[DEBUG] Top K: {k}")
    try:
        results = qdrant_search_tool_instance.search(query=query, filters=filters, k=k)
        print(f"[DEBUG] Raw results: {results}")
        return results
    except Exception as e:
        print(f"[ERROR] Qdrant search failed: {e}")
        return []


In [16]:
# For LangChain agent compatibility
qdrant_search_tool = Tool(
    name="qdrant_search",
    func=wrapped_qdrant_search,
    description="""Perform hybrid semantic + metadata searches over our SuperVator startup knowledge base 🔍🚀

• **Semantic Retrieval** – Leverage OpenAI embeddings + Qdrant’s cosine‐distance vector index to find the most contextually relevant companies for ANY natural-language query (“emerging fintech players”, “best agritech startups”, etc.).

• **Metadata & Keyword Filters** – Narrow down results by exact or fuzzy matching on structured fields:
    – ▶️ *Categorical*: `state`, `industry_sector`, `hiring_status`, `tech_stack`, `founders`, etc.  
    – ▶️ *Numeric Ranges*: `year_founded`, `total_funding_raised_inr`, `number_of_employees_current`, etc. (supports `gte`/`lte` filters)

• **Fully Hybrid** – Mix & match: e.g. “top funded SaaS companies in Delhi founded after 2015”  
simply by passing your free-text query plus a `filters` dict:"""
   
)

---

**This notebook only imports and registers the Qdrant search tool. All implementation logic lives in the qdrant_server_tool.py script.**